In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

# Ex 3-2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

device = torch.device("cuda")

def train(model, train_loader, test_loader, criterion, optimizer, epoch):
  train_loss = []
  train_acc = []
  test_loss = []
  test_acc = []
  for n in range(epoch):
    model.train()

    train_loss_step = []
    train_acc_step = []
    for i, (x, y) in enumerate(train_loader):
      optimizer.zero_grad() 
      x = x.to(device)
      y = y.to(device)

      pred = model(x)
      loss = criterion(pred, y)
      loss.backward()
      optimizer.step()

      train_loss_step.append(float(loss))
      acc = accuracy_score(y.to('cpu').detach().numpy().copy(), 
                          pred.argmax(dim=1).to('cpu').detach().numpy().copy())
      train_acc_step.append(acc)
    train_loss.append(np.mean(train_loss_step))
    train_acc.append(np.mean(train_acc_step))

    with torch.no_grad():
      model.eval()
      test_loss_step = []
      test_acc_step = []
      for i ,(x, y) in enumerate(test_loader):
        x = x.to(device)
        y = y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        test_loss_step.append(float(loss))
        acc = accuracy_score(y.to('cpu').detach().numpy().copy(), 
                         pred.argmax(dim=1).to('cpu').detach().numpy().copy())
        test_acc_step.append(acc)
  
    test_loss.append(np.mean(test_loss_step))
    test_acc.append(np.mean(test_acc_step))
    print("%i: %f, %f, %f, %f" % (n, train_acc[-1], test_acc[-1], train_loss[-1], test_loss[-1]))

In [3]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5), 
                                transforms.ToTensor(),transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
                                transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1024, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1024, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9,weight_decay=0.0005)

train(model, train_loader, test_loader, criterion, optimizer, epoch=50)

0: 0.181208, 0.255267, 2.457356, 2.782020
1: 0.314854, 0.346002, 1.949112, 1.864619
2: 0.376420, 0.389499, 1.753448, 1.691441
3: 0.411500, 0.411312, 1.643805, 1.643843
4: 0.435049, 0.433349, 1.596156, 1.590451
5: 0.460253, 0.451834, 1.508606, 1.521692
6: 0.487091, 0.460900, 1.431386, 1.507439
7: 0.506198, 0.475484, 1.383362, 1.474326
8: 0.520021, 0.479781, 1.347545, 1.454264
9: 0.537190, 0.491199, 1.301254, 1.445751
10: 0.547034, 0.496636, 1.278289, 1.400014
11: 0.566779, 0.508586, 1.220201, 1.390236
12: 0.575545, 0.509690, 1.187587, 1.406466
13: 0.587239, 0.504695, 1.160286, 1.402573
14: 0.603188, 0.521875, 1.134244, 1.405222
15: 0.615672, 0.530744, 1.094333, 1.341418
16: 0.627187, 0.534935, 1.060572, 1.335700
17: 0.634167, 0.535756, 1.039866, 1.335290
18: 0.645769, 0.536685, 1.012646, 1.334430
19: 0.652959, 0.522752, 0.988430, 1.404160
20: 0.663566, 0.543872, 0.964292, 1.336307
21: 0.677244, 0.542883, 0.922104, 1.377673
22: 0.686525, 0.543732, 0.892565, 1.337854
23: 0.696278, 0.53989

#Ex 3-3

## Res2Net

In [ ]:
!git clone https://github.com/gasvn/Res2Net.git
from Res2Net.res2net import res2net50

Cloning into 'Res2Net'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 124 (delta 2), reused 0 (delta 0), pack-reused 118
Receiving objects: 100% (124/124), 61.74 KiB | 15.43 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [ ]:
model = res2net50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9,weight_decay=0.00005)

train(model, train_loader, test_loader, criterion, optimizer, epoch=50)

0: 0.162018, 0.246405, 2.755706, 3.244640
1: 0.304937, 0.330415, 1.967387, 1.910489
2: 0.370903, 0.397188, 1.782244, 1.696594
3: 0.411257, 0.414039, 1.656783, 1.634610
4: 0.441096, 0.435511, 1.554704, 1.586944
5: 0.471061, 0.464535, 1.473918, 1.478753
6: 0.487470, 0.477244, 1.429020, 1.461534
7: 0.502116, 0.474013, 1.383595, 1.468254
8: 0.517639, 0.500927, 1.358420, 1.410050
9: 0.532501, 0.489232, 1.309177, 1.432111
10: 0.549727, 0.496592, 1.263728, 1.445346
11: 0.559702, 0.514925, 1.241766, 1.375673
12: 0.573936, 0.518742, 1.197854, 1.341124
13: 0.587009, 0.529255, 1.166924, 1.337309
14: 0.597426, 0.524984, 1.140040, 1.348646
15: 0.612487, 0.529048, 1.100454, 1.345031
16: 0.613499, 0.530052, 1.090568, 1.339492
17: 0.624734, 0.527290, 1.065778, 1.348499
18: 0.636730, 0.542498, 1.033571, 1.323721
19: 0.645654, 0.541067, 1.004542, 1.343962
20: 0.652121, 0.537331, 0.988137, 1.320251
21: 0.664855, 0.549755, 0.954057, 1.333108
22: 0.674183, 0.550273, 0.933556, 1.324518
23: 0.680065, 0.55049

## Squeeze-and-Excitation

In [4]:
import torch.hub
model = torch.hub.load('moskomule/senet.pytorch', 'se_resnet50', pretrained=False,)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9,weight_decay=0.00005)

train(model, train_loader, test_loader, criterion, optimizer, epoch=50)

Downloading: "https://github.com/moskomule/senet.pytorch/archive/master.zip" to /root/.cache/torch/hub/master.zip



0: 0.216372, 0.283317, 2.137071, 2.038299
1: 0.355717, 0.391255, 1.768723, 1.681448
2: 0.417319, 0.436155, 1.603598, 1.561689
3: 0.454265, 0.461926, 1.492467, 1.483735
4: 0.483917, 0.479951, 1.413934, 1.444403
5: 0.510637, 0.490830, 1.353255, 1.426402
6: 0.535843, 0.495402, 1.283929, 1.408292
7: 0.555660, 0.509088, 1.235617, 1.365070
8: 0.581146, 0.522826, 1.171504, 1.345470
9: 0.595596, 0.530465, 1.128875, 1.333403
10: 0.613501, 0.526539, 1.080595, 1.352819
11: 0.634636, 0.541671, 1.025751, 1.321056
12: 0.648134, 0.552714, 0.987825, 1.301567
13: 0.664278, 0.544031, 0.951032, 1.325998
14: 0.675440, 0.557462, 0.916771, 1.299776
15: 0.687784, 0.559166, 0.890604, 1.331160
16: 0.703233, 0.560316, 0.844460, 1.310197
17: 0.713805, 0.566317, 0.817680, 1.306282
18: 0.724940, 0.572393, 0.786639, 1.304094
19: 0.738725, 0.563560, 0.750076, 1.348318
20: 0.746825, 0.567130, 0.726434, 1.329974
21: 0.759074, 0.578657, 0.696285, 1.332272
22: 0.768786, 0.579823, 0.672258, 1.333863
23: 0.777059, 0.5743